# **Laboratorio 4**

Angélica Ortiz - 202222480 
<br>
María José Amorocho - 202220179

# Importaciones

In [2]:
#importaciones
import numpy as np
from tabulate import tabulate

# Problema 1 - Implementación del Método Simplex Estándar

## Implementación

In [3]:
def simplex_con_iteraciones(A, b, c, imprimir=True, modo='max'):
    """
    Implementa el Método Simplex y retorna los valores finales 
    de todas las variables (originales y de holgura).
    
    Máx  z = c^T x
    s.a. A x <= b
         x >= 0

    Args:
    A :  Matriz de coeficientes de las restricciones (m x n)
    b :  Vector de términos independientes (m,1)
    c :  Vector de coeficientes de la función objetivo (n,)
    imprimir : Si es True, se imprime la tabla y detalles en cada iteración
    modo: si es problema de maximización (max) o minimización (min)

    Returns:
    matriz_simplex : Matriz después de toads las iteraciones
    z_opt : Valor óptimo de la función objetivo
    """
    # Establecer numero de restricciones y de variables
    m, n = A.shape
    
    # Si es un problema de minimización, convertirlo a maximización
    if modo == 'min':
        c = -c
    
    # Establecer matriz de identidad para variables de holgura
    I = np.eye(m)
    
    # Construccion de la matriz A con las variables de holgura incluidas
    A_ext = np.hstack([A, I]) #matriz de tamaño m x (n+m)
    
    #Extender el vector c para incluir las variables de holgura
    c_ext = np.concatenate([c, np.zeros(m)])
    
    # Construir la tabla: dimensiones (m+1) x (n+m+1)
    # (m+1) filas > m restricciones + 1 fila de función objetivo
    # (n+m+1) columnas > n+m variables (originales y de holgura) + 1columna para terminos independientes (b)
    matriz_simplex = np.zeros((m+1, n+m+1))
    
    # Llenar las filas de restricciones
    for i in range(m):
        matriz_simplex[i, :-1] = A_ext[i]
        matriz_simplex[i, -1]  = b[i]
    
    # Fila de la función objetivo (utilizando -c para la maximización)
    matriz_simplex[-1, :-1] = -c_ext
    
    # iniciar las iteraciones
    iteracion = 0
    while True:
        if imprimir:
            print(f"Iteración {iteracion}:")
            print(tabulate(matriz_simplex, floatfmt=".3f", tablefmt="grid"))
            print("-" * 50)
        
        # 1. Buscar la columna pivote (más negativa en la fila de la función objetivo)
        fila_objetivo = matriz_simplex[-1, :-1]
        col_in = np.argmin(fila_objetivo)
        valor_min = fila_objetivo[col_in]
        
        # si todos los valores en la fila de la función objetivo son positivos se ha encontrado el optimo
        if valor_min >= 0:
            if imprimir:
                print("Óptimo alcanzado.")
            break  # Solución óptima alcanzada
        
        # 2. Seleccionar la fila pivote usando el cociente mínimo
        columna_pivote = matriz_simplex[:-1, col_in]
        b_column = matriz_simplex[:-1, -1]
        
        # calcular las divisiones o razones
        razones = []
        for i in range(m):
            if columna_pivote[i] > 1e-15: # manejar divisiones por cero o muy pequeñas
                razones.append(b_column[i] / columna_pivote[i])
            else:
                razones.append(np.inf)
                
         # La fila con la razón mínima es la que sale de la base
        fila_out = np.argmin(razones)
        if razones[fila_out] == np.inf:
            # Problema no acotado > cuando todas las razones calculadas dan infinito
            raise ValueError("El problema es no acotado.")
        
        if imprimir:
            print(f"Columna que entra: {col_in+1}")
            print(f"Fila que sale: {fila_out+1}")
            print(f"Valor pivote: {matriz_simplex[fila_out, col_in]}")
        
        # 3. normalizar la fila pivote para que el elemento pivote sea 1
        pivote = matriz_simplex[fila_out, col_in]
        matriz_simplex[fila_out, :] /= pivote
        
        # Copnvetir en ceros en el resto de la columna pivote
        for i in range(m+1):
            if i != fila_out:
                factor = matriz_simplex[i, col_in]
                matriz_simplex[i, :] -= factor * matriz_simplex[fila_out, :]
        
        iteracion += 1
        
    # Valor óptimo obtenido en la tabla
    z_opt = matriz_simplex[-1, -1]
    # Para problemas de minimización, revertir el signo del óptimo
    if modo == 'min':
        z_opt = -z_opt
    
    return matriz_simplex, z_opt

def extraer_resultados_simplex(matriz_simplex, A):
    """
    Extrae los resultados de las variables de una matriz final

    Args:
        matriz_simplex: matriz resultante que dontiene el valor objetivo
        A: Matriz de restricciones

    Returns:
        resultado: diccionario con los valores de las variables
    """
    m, n = A.shape
    
    # Extraer la solución óptima > se revisan todas las columnas (originales + holgura)
    x_opt = np.zeros(n + m)
    for col_index in range(n + m):
        # Extraer la columna correspondiente a las filas de restricciones
        col = matriz_simplex[:m, col_index]
        # verificar si es columna canónica (un 1 y el resto 0's)
        if np.count_nonzero(col) == 1 and np.isclose(np.max(col), 1.0):
            row_index = np.argmax(col)
            x_opt[col_index] = matriz_simplex[row_index, -1]
    
    # diccionario para diferenciar variables originales y de holgura
    resultados = {}
    for i in range(n):
        resultados[f'x{i+1}'] = x_opt[i]
    for i in range(m):
        resultados[f'x{n+i+1}'] = x_opt[n + i]
    
    return resultados
    
    

## Primera ejecución

In [4]:
# Ejemplo:
# Máx z = 3x1 + 2x2 + 5x3
# Restricciones
#   x1  +  x2  +  x3 <= 100
#   2x1 +  x2  +  x3 <= 150
#   x1  +  4x2 + 2x3 <= 80
# x1, x2, x3 >= 0

A = np.array([
    [1, 1, 1],
    [2, 1, 1],
    [1, 4, 2]
], dtype=float)

b = np.array([100, 150, 80], dtype=float)
c = np.array([3, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=True, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)

print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Iteración 0:
+--------+--------+--------+--------+--------+--------+---------+
|  1.000 |  1.000 |  1.000 |  1.000 |  0.000 |  0.000 | 100.000 |
+--------+--------+--------+--------+--------+--------+---------+
|  2.000 |  1.000 |  1.000 |  0.000 |  1.000 |  0.000 | 150.000 |
+--------+--------+--------+--------+--------+--------+---------+
|  1.000 |  4.000 |  2.000 |  0.000 |  0.000 |  1.000 |  80.000 |
+--------+--------+--------+--------+--------+--------+---------+
| -3.000 | -2.000 | -5.000 | -0.000 | -0.000 | -0.000 |   0.000 |
+--------+--------+--------+--------+--------+--------+---------+
--------------------------------------------------
Columna que entra: 3
Fila que sale: 3
Valor pivote: 2.0
Iteración 1:
+--------+--------+-------+-------+-------+--------+---------+
|  0.500 | -1.000 | 0.000 | 1.000 | 0.000 | -0.500 |  60.000 |
+--------+--------+-------+-------+-------+--------+---------+
|  1.500 | -1.000 | 0.000 | 0.000 | 1.000 | -0.500 | 110.000 |
+--------+--------+--

## Segunda ejecución

En esta ejecución, los coeficientes de la función objetivo son modificados con el fin de, posteriormente, hacer un análisis breve de sensibilidad

In [5]:
# incrementando x1 en 1
c = np.array([4, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 310.0


In [6]:
# incrementando x2 en 1
c = np.array([3, 3, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 236.667


In [7]:
# incrementando x3 en 1
c = np.array([3, 2, 6], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 240.0


## Tercera ejecución

En esta ejecución los términos independientes que afectan la solución son modificados

In [8]:
# Incrementando primer termino independiente en 1
b = np.array([101, 150, 80], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 240.0


In [9]:
# Incrementando segundo termino independiente en 1
b = np.array([100, 151, 80], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 240.0


In [10]:
# Incrementando tercer termino independiente en 1
b = np.array([100, 150, 81], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Valor óptimo de la función objetivo: 243.0


# Problema 2 - Implementación del Método Simplex Dual Phase

# Problema 3 - Comparación de Rendimiento con GLPK/Pyomo

# Problema 4 - Análisis de Sensibilidad en Programación Lineal

## Análisis de Sensibilidad para Coeficientes de la Función Objetivo

### Implementación

In [ ]:
def analisis_sensibilidad_coeficientes(matriz_simplex, c, n, m, modo='max', imprimir=True):
    """
    Realiza el análisis de sensibilidad de los coeficientes de la función objetivo
    
    Args:
    matriz_simplex : matriz final del Método Simplex > de dimensión (m+1) x (n+m+1)
    c : Coeficientes originales de la función objetivo (tamaño n).
    n : numero de variables originales (excluyendo holgura).
    m : numero de restricciones.
    modo : 'max'si el problema es de maximización,'min'si es de minización
    imprimir : Si True, imprime resultados

    Return:
    dict_sensibilidad : diccionario con rango de variacion de las variables (basicas y no basicas)
    """
    # Identificar variables básicas y no básicas a partir de la submatriz de restricciones
    submatriz_restricciones = matriz_simplex[:m, :n+m]  # m filas, (n+m) columnas (sin la columna RHS)
    
    cols_basicas = []
    for j in range(n+m):
        col_j = submatriz_restricciones[:, j]
        # Chequeo vector identidad en la submatriz
        if np.count_nonzero(col_j) == 1:
            fila_1 = np.where(abs(col_j - 1) < 1e-12)[0]
            if len(fila_1) == 1:
                i_fila = fila_1[0]
                # Verificar ceros en el resto
                if all(abs(x) < 1e-12 for idx, x in enumerate(col_j) if idx != i_fila):
                    cols_basicas.append(j)

    todas_las_cols = list(range(n+m))
    cols_no_basicas = [j for j in todas_las_cols if j not in cols_basicas]

    
    # Extender c con ceros para las variables de holgura
    c_extended = np.concatenate([c, np.zeros(m)]) #c_extended[j] es el "coeficiente actual" c_j^actual de la columna j
    
   
    # Obtener costos reducidos (r_j) desde la última fila de la tabla final 
    fila_objetivo = matriz_simplex[-1, :n+m]  # fila_objetivo son las primeras (n+m) columnas de la última fila


    if modo == 'max':
        reduced_costs = -fila_objetivo

    # Calcular los rangos de estabilidad para cada variable
    dict_sensibilidad = {'basicas': {}, 'no_basicas': {}}

    # Para cada variable NO básica x_j
    for j in cols_no_basicas:
        var_name = f"x{j+1}"
        r_j = reduced_costs[j]
        c_j_actual = c_extended[j]


        if r_j > 0:
            # si r_j > 0 c_j puede disminuir hasta que r_j se vuelva 0
            # intevalos -> - r_j, infinito
            c_j_min = c_j_actual - r_j
            c_j_max = float('inf')
            interpretacion = "Si c_j baja a c_j_min, x_j podría entrar en la base"
        else:  # r_j < 0
            # c_j puede aumentar hasta que r_j se vuelva 0
            #intervalos -> -infinito, c_j_actual - r_j
            c_j_min = -float('inf')
            c_j_max = c_j_actual - r_j
            interpretacion = "Si c_j sube a c_j_max, x_j podría entrar en la base"

        dict_sensibilidad['no_basicas'][var_name] = {
            'c_j_actual': c_j_actual,
            'c_j_min': c_j_min,
            'c_j_max': c_j_max,
            'r_j': r_j,
            'interpretacion': interpretacion
        }

    if modo == 'max':
        reduced_costs = -reduced_costs
    
     # Para cada variable BÁSICA x_i
    for j in cols_basicas:
        var_name = f"x{j+1}"
        c_j_actual = c_extended[j]
        col_j = submatriz_restricciones[:, j]

        # Buscar la fila i donde col_j tiene 1
        fila_i_arr = np.where(abs(col_j - 1) < 1e-12)[0]
        if len(fila_i_arr) != 1:
            # Caso sin solución
            dict_sensibilidad['basicas'][var_name] = {
                'c_j_actual': c_j_actual,
                'c_j_min': None,
                'c_j_max': None,
                'interpretacion': "No se identificó una fila pivot única"
            }
            continue

        i = fila_i_arr[0]
        # Coeficientes de la fila i en las columnas [0..n+m)
        fila_i_coeffs = matriz_simplex[i, :n+m]

        valores_inc = []  # candidatos para delta c_i^+
        valores_dec = []  # candidatos para delta c_i^-

        for k in cols_no_basicas:
            a_ik = fila_i_coeffs[k]
            r_k = reduced_costs[k]

            if abs(a_ik) < 1e-12:
                continue

            if a_ik < 0:
                val_posible = -r_k / a_ik
                if val_posible >= 0:
                    valores_inc.append(val_posible)
            else:  # a_ik > 0
                val_posible = r_k / a_ik
                if val_posible >= 0:
                    valores_dec.append(val_posible)

        # Tomar mínimos
        if len(valores_inc) > 0:
            delta_c_i_plus = min(valores_inc)
        else:
            delta_c_i_plus = float('inf')

        if len(valores_dec) > 0:
            delta_c_i_minus = min(valores_dec)
        else:
            delta_c_i_minus = float('inf')

        # Rango para c_i:
        c_j_min = c_j_actual - delta_c_i_minus
        c_j_max = c_j_actual + delta_c_i_plus

        dict_sensibilidad['basicas'][var_name] = {
            'c_j_actual': c_j_actual,
            'c_j_min': c_j_min,
            'c_j_max': c_j_max,
            'incremento_permitido': delta_c_i_plus,
            'decremento_permitido': delta_c_i_minus
        }

    # Imprimir resultados
    
    if imprimir:
        headers = ['variable', 'intervalo', 'Permisible reducir', 'actual', 'Permisible aumentar']
        print("\n=== Análisis de Sensibilidad de Coeficientes ===")
           
        tabla = []
        for var_no_basica, info in dict_sensibilidad["no_basicas"].items():
            fila = []
            intervalo = f"{info['c_j_min']:.2f} < {var_no_basica} < {info['c_j_max']:.2f}"
            fila.append(var_no_basica)
            fila.append(intervalo)
            fila.append(info['c_j_min'])
            fila.append(info['c_j_actual'])
            fila.append(info['r_j']*-1)
            tabla.append(fila)
            
        print('Variables NO básicas \n' + tabulate(tabla, headers=headers, tablefmt='grid'))
            
        print("\nVariables BÁSICAS:")
        tabla2 = []
        for var_name, info in dict_sensibilidad['basicas'].items():
            fila = []
            fila.append(var_name)
            intervalo = f"{info['c_j_min']:.2f} < {var_name} < {info['c_j_max']:.2f}"
            fila.append(intervalo)
            fila.append(info['decremento_permitido'])
            fila.append(info['c_j_actual'])
            fila.append(info['incremento_permitido'])
            tabla2.append(fila)
            
        print(tabulate(tabla2, headers=headers, tablefmt='grid'))
    return dict_sensibilidad


### Ejecución

In [79]:
# Máx z = 3x1 + 2x2 + 5x3
# Restricciones
#   x1  +  2x2  + x3 <= 430
#   3x1 + 2x3  <= 460
#   x1 + 4x2  <= 420
# x1, x2 >= 0

A = np.array([
    [1, 2, 1],
    [3, 0, 2],
    [1, 4, 0]
], dtype=float)

b = np.array([430, 460, 420], dtype=float)
c = np.array([3, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)

print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

# Dimensiones de A
m, n = A.shape
sensibilidad = analisis_sensibilidad_coeficientes(matriz_simplex, c, n, m, modo='max', imprimir=True)



Valor final de las variables:
> x1: 0.0
> x2: 100.0
> x3: 230.0
> x4: 0.0
> x5: 0.0
> x6: 20.0
Valor óptimo de la función objetivo: 1350.0

=== Análisis de Sensibilidad de Coeficientes ===
Variables NO básicas 
+------------+------------------+----------------------+----------+-----------------------+
| variable   | intervalo        |   Permisible reducir |   actual |   Permisible aumentar |
+============+==================+======================+==========+=======================+
| x1         | -inf < x1 < 7.00 |                 -inf |        3 |                     4 |
+------------+------------------+----------------------+----------+-----------------------+
| x4         | -inf < x4 < 1.00 |                 -inf |        0 |                     1 |
+------------+------------------+----------------------+----------+-----------------------+
| x5         | -inf < x5 < 2.00 |                 -inf |        0 |                     2 |
+------------+------------------+-------------------

## Análisis de Sensibilidad para Términos Independientes

### Implementación

In [ ]:
def analisis_sensibilidad_independientes(matriz_simplex, A, b, c, modo='max', imprimir=True):
    """
    Análisis de sensibilidad para términos independientes y shadow prices.

    Args:
    matriz_simplex : Matriz final después de Simplex > dimensiones (m+1, n+m+1)
    A : matriz original de restricciones.
    b : Vector de términos independientes originales.
    c : Vector de coeficientes de la función objetivo.
    modo : 'max' si el problema es de maximización,'min' si el problema es de minimización
    imprimir : Si True, imprime resultados

    Returns:
    dict_sens : diccionario de sensilidad con shadow_prices y rangos_b (rangos en los que se puede mover el coeficiente, 
    su mínimo y su maximo)
    """
    m, n = A.shape

    # Construir A_ext = [A | I_m]
    A_ext = np.hstack([A, np.eye(m)])

    # Identificar columnas básicas en la submatriz de restricciones
    subR = matriz_simplex[:m, :n+m]
    cols_basicas = []
    for j in range(n+m):
        col = subR[:, j]
        if np.count_nonzero(col)==1:
            fila = np.where(abs(col-1)<1e-12)[0]
            if len(fila)==1 and all(abs(col[k])<1e-12 for k in range(m) if k!=fila[0]):
                cols_basicas.append(j)

    # Base B y su inversa
    B = A_ext[:, cols_basicas]
    B_inv = np.linalg.inv(B)

    # Shadow prices: y* = c_B^T B^{-1}
    c_ext = np.concatenate([c, np.zeros(m)])
    c_B   = c_ext[cols_basicas]
    y_star = c_B @ B_inv

    # si es minimización > multiplicar por -1
    if modo=='min':
        y_star = -y_star

    # Cálculo de rangos para cada b_i
    x_B = B_inv @ b  # solución básica actual
    rangos = []
    for i in range(m):
        beta_col = B_inv[:, i]  # beta_{k,i} para k=0..m-1

        # Acumular cotas
        cotas_inferiores = []  # delta_b >= …
        cotas_superiores = []  # delta_b <= …

        for k in range(m):
            beta = beta_col[k]
            xBk = x_B[k]
            if abs(beta)<1e-12:
                continue
            ratio = - xBk / beta
            if beta>0:
                cotas_inferiores.append(ratio)
            else:
                cotas_superiores.append(ratio)

        # Si no hay cotas de un tipo se toma infinito
        delta_b_minus = max(cotas_inferiores) if cotas_inferiores else -np.inf
        delta_b_plus  = min(cotas_superiores) if cotas_superiores else  np.inf

        b_actual = b[i]
        rangos.append({
            'restricción': i+1,
            'b_i_actual': b_actual,
            'delta_b_i_menos': delta_b_minus,
            'delta_b_i_mas':  delta_b_plus,
            'b_i_min': b_actual + delta_b_minus,
            'b_i_max': b_actual + delta_b_plus
        })

    # Imprimir resultados
    if imprimir:
        print("=== Shadow prices (y*) ===")
        for i, y in enumerate(y_star, start=1):
            print(f" y_{i}* = {y}")
        print("\n=== Rangos de estabilidad ===")
        
        
        # Datos para la tabla
        tabla_rangos = []
        i = 1
        for r in rangos:
            intervalo = f"{r['delta_b_i_menos']:.3f} < b{i} < {r['delta_b_i_mas']:.3f}"
            fila = [
                r['restricción'],
                intervalo,
                f"{r['b_i_min']:.3f}",
                f"{r['b_i_actual']:.3f}",
                f"{r['b_i_max']:.3f}"
            ]
            tabla_rangos.append(fila)
            i += 1
        
        # Encabezados de la tabla
        headers = ["Restricción", "Intervalo de factibilidad", "Mínimo", "Actual", "Máximo"]
        
        # Imprimir tabla
        from tabulate import tabulate
        print(tabulate(tabla_rangos, headers=headers, tablefmt="grid", floatfmt=".3f"))

    return {
        'shadow_prices': y_star,
        'rangos_b': rangos
    }


### Ejecución

In [87]:
# Máx z = 3x1 + 2x2 + 5x3
# Restricciones
#   x1  +  2x2  + x3 <= 430
#   3x1 + 2x3  <= 460
#   x1 + 4x2  <= 420
# x1, x2 >= 0

A = np.array([
    [1, 2, 1],
    [3, 0, 2],
    [1, 4, 0]
], dtype=float)

b = np.array([430, 460, 420], dtype=float)
c = np.array([3, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)

print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

resultado_rhs = analisis_sensibilidad_independientes(matriz_simplex, A, b, c, modo='max',imprimir=True)


Valor final de las variables:
> x1: 0.0
> x2: 100.0
> x3: 230.0
> x4: 0.0
> x5: 0.0
> x6: 20.0
Valor óptimo de la función objetivo: 1350.0
=== Shadow prices (y*) ===
 y_1* = 1.0
 y_2* = 2.0
 y_3* = 0.0

=== Rangos de estabilidad ===
+---------------+-----------------------------+----------+----------+----------+
|   Restricción | Intervalo de factibilidad   |   Mínimo |   Actual |   Máximo |
+===============+=============================+==========+==========+==========+
|             1 | -200.000 < b1 < 10.000      |  230.000 |  430.000 |  440.000 |
+---------------+-----------------------------+----------+----------+----------+
|             2 | -20.000 < b2 < 400.000      |  440.000 |  460.000 |  860.000 |
+---------------+-----------------------------+----------+----------+----------+
|             3 | -20.000 < b3 < inf          |  400.000 |  420.000 |  inf     |
+---------------+-----------------------------+----------+----------+----------+
